<a href="https://colab.research.google.com/github/hargurjeet/LJMU_Thesis/blob/main/TabToText_Exp22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import openai
openai.api_key = ''

In [3]:
travel_data = pd.read_csv('/content/Customertravel.csv')
travel_data.head()

,Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot,Target
0,34,No,Middle Income,6,No,Yes,0
1,34,Yes,Low Income,5,Yes,No,1
2,37,No,Middle Income,3,Yes,No,0
3,30,No,Middle Income,2,No,No,0
4,30,No,Low Income,1,No,No,0


# Experiment 1

# Experiment 2

In [4]:
dummy_df = travel_data.head()
dummy_df

,Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot,Target
0,34,No,Middle Income,6,No,Yes,0
1,34,Yes,Low Income,5,Yes,No,1
2,37,No,Middle Income,3,Yes,No,0
3,30,No,Middle Income,2,No,No,0
4,30,No,Low Income,1,No,No,0


In [ ]:
def row_to_textual_data(row):
    row_str = ', '.join([f"{col}: {val}" for col, val in row.items()])
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"""A Tour & Travels Company Wants To Predict Whether A Customer Will Churn Or Not Based On Indicators Given Below.
                                            Convert the following row to a detailed textual description: {row_str}"""}
        ]
    )
    return response['choices'][0]['message']['content']

# Apply the function to each row of the DataFrame and save the results in a new DataFrame
dummy_df['TextualData'] = dummy_df.apply(row_to_textual_data, axis=1)

# Convert the Series to a DataFrame
# textual_df = pd.DataFrame(textual_data, columns=['TextualData'])
dummy_df

In [7]:
import pandas as pd
import openai
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def get_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response['data'][0]['embedding']

dummy_df['Embedding'] = dummy_df['TextualData'].apply(get_embedding)

<ipython-input-7-d60bcd6ebc35>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy_df['Embedding'] = dummy_df['TextualData'].apply(get_embedding)


In [ ]:
dummy_df['Embedding'][0]

In [ ]:
# Prepare the data for training
X = pd.DataFrame(df['Embedding'].tolist())  # Convert list of embeddings to DataFrame
y = df['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Display the DataFrame with the embeddings and the target column
df[['Embedding', 'Target']])